## Deliverable 2. Create a Customer Travel Destinations Map.

In [24]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [41]:
# 1. Import the WeatherPy_database.csv file. 

city_data_df = pd.read_csv('../Weather_Database/WeatherPy_Database.csv')
city_data_df.head()

,City_ID,City,Lat,Lng,Current Description,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,Caravelas,-17.7125,-39.2481,scattered clouds,75.43,94,40,6.24
1,1,Nikolskoye,59.7035,30.7861,light snow,24.69,97,97,5.93
2,2,Pisco,-13.7000,-76.2167,clear sky,69.85,73,0,3.44
3,3,Bredasdorp,-34.5322,20.0403,broken clouds,59.29,75,52,6.40
4,4,Kapaa,22.0752,-159.3190,clear sky,78.78,71,0,1.01


In [42]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 

min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 60
What is the maximum temperature you would like for your trip? 80


In [43]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.

clean_cites_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
clean_cites_df.head(10)

,City_ID,City,Lat,Lng,Current Description,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,Caravelas,-17.7125,-39.2481,scattered clouds,75.43,94,40,6.24
2,2,Pisco,-13.7000,-76.2167,clear sky,69.85,73,0,3.44
4,4,Kapaa,22.0752,-159.3190,clear sky,78.78,71,0,1.01
5,5,Mataura,-46.1927,168.8643,broken clouds,66.47,24,75,7.23
6,6,Rikitea,-23.1203,-134.9692,overcast clouds,77.86,74,92,13.80
8,8,New Norfolk,-42.7826,147.0587,overcast clouds,71.91,52,99,4.00
16,16,Amapa,1.0000,-52.0000,moderate rain,70.79,99,80,1.32
17,17,Izazi,-7.2000,35.7333,heavy intensity rain,68.16,97,100,3.53
19,19,Saint-Philippe,-21.3585,55.7679,clear sky,79.11,76,0,6.69
22,22,Coahuayana,18.7333,-103.6833,broken clouds,69.82,84,53,3.09


In [44]:
# 4a. Determine if there are any empty rows.
clean_cites_df.count()

City_ID                239
City                   239
Lat                    239
Lng                    239
Current Description    239
Max Temp               239
Humidity               239
Cloudiness             239
Wind Speed             239
dtype: int64

In [45]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.

In [47]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_cites_df[["City", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Caravelas,75.43,scattered clouds,-17.7125,-39.2481,
2,Pisco,69.85,clear sky,-13.7000,-76.2167,
4,Kapaa,78.78,clear sky,22.0752,-159.3190,
5,Mataura,66.47,broken clouds,-46.1927,168.8643,
6,Rikitea,77.86,overcast clouds,-23.1203,-134.9692,
8,New Norfolk,71.91,overcast clouds,-42.7826,147.0587,
16,Amapa,70.79,moderate rain,1.0000,-52.0000,
17,Izazi,68.16,heavy intensity rain,-7.2000,35.7333,
19,Saint-Philippe,79.11,clear sky,-21.3585,55.7679,
22,Coahuayana,69.82,broken clouds,18.7333,-103.6833,


In [48]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame  
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [54]:
# 7. Drop the rows where there is no Hotel Name.
import numpy as np

hotel_df["Hotel Name"].replace('',np.nan, inplace=True)
hotel_df.dropna(subset=["Hotel Name"], inplace=True)
clean_hotel_df = hotel_df
clean_hotel_df.head()

,City,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Caravelas,75.43,scattered clouds,-17.7125,-39.2481,Pousada dos Navegantes
2,Pisco,69.85,clear sky,-13.7000,-76.2167,Hotel San Isidro Oficial
4,Kapaa,78.78,clear sky,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort
5,Mataura,66.47,broken clouds,-46.1927,168.8643,Ellie's Villa
6,Rikitea,77.86,overcast clouds,-23.1203,-134.9692,People ThankYou


In [58]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_Vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [59]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>City</dt><dd>{City}</dd>
<dt>Current Description </dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations_df = clean_hotel_df[["Lat", "Lng"]]

In [60]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations_df, dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations_df, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))